In [1]:
"""An example of how to generate a structured grid dataset using
numpy arrays.  Also shown is a way to visualize this data with
the mayavi2 application.

The script can be run like so::

  $ mayavi2 -x structured_grid.py

Alternatively, it can be run as::

  $ python structured_grid.py


"""

# Authors: Eric Jones <eric at enthought dot com>
#          Prabhu Ramachandran <prabhu at aero dot iitb dot ac dot in>
# Copyright (c) 2007, Enthought, Inc.
# License: BSD style.

import numpy as np
from numpy import cos, sin, pi
from tvtk.api import tvtk
from mayavi.scripts import mayavi2
from PyQt5 import QtCore
import sip
import vtk

In [2]:
from mayavi import mlab
%gui qt
print("ok")

ok


In [3]:
# clears the field
mlab.clf()
# test the 3d plotting function
mlab.test_plot3d()

In [4]:
class grid:
    def __init__(self, ngrid_cart, ngrid_tor, R0):
        self.R, self.theta, self.phi = np.meshgrid(np.linspace(1e-2, 1,ngrid_tor), 
                                                   np.linspace(0,2*np.pi, ngrid_tor), 
                                                   np.linspace(0,2*np.pi, 2*ngrid_tor),
                                                   indexing='ij')
        self.cart_x, self.cart_y, self.cart_z = np.meshgrid(np.linspace(-3,3, ngrid_cart),
                                                            np.linspace(-3,3, ngrid_cart),
                                                            np.linspace(-1,1, ngrid_cart),
                                                            indexing='ij')
        self.tor_x = (R0 + self.R*np.cos(self.theta))*np.cos(self.phi)
        self.tor_y = (R0 + self.R*np.cos(self.theta))*np.sin(self.phi)
        self.tor_z = self.R*np.sin(self.theta)
        
        self.cartflat = np.zeros((ngrid_cart**3, 3))
        self.cartflat[:,0] = self.cart_x.flatten()
        self.cartflat[:,1] = self.cart_y.flatten()
        self.cartflat[:,2] = self.cart_z.flatten()
        
        self.torflat = np.zeros((2*ngrid_tor**3, 3))
        self.torflat[:,0] = self.tor_x.flatten()
        self.torflat[:,1] = self.tor_y.flatten()
        self.torflat[:,2] = self.tor_z.flatten()
    def plotgrid(self):
        fig_gridscat = plt.figure()
        ax_gridscat = fig_gridscat.add_subplot(projection='3d')
        ax_gridscat.scatter(self.cart_x, self.cart_y, self.cart_z, marker='.')
        ax_gridscat.scatter(self.tor_x, self.tor_y, self.tor_z, marker='x')
        ax_gridscat.set_xlabel("$x[\\mathrm{m}]$")
        ax_gridscat.set_ylabel("$y[\\mathrm{m}]$")
        ax_gridscat.set_zlabel("$z[\\mathrm{m}]$")
    def delaunay_tessalation(self):
        tri = Delaunay(self.torflat)
        idx_simplices = tri.simplices
        coord_simplices = self.torflat[idx_simplices]
        return tri, idx_simplices, coord_simplices

In [5]:
def generate_annulus(r=None, theta=None, z=None):
    """ Generate points for structured grid for a cylindrical annular
        volume.  This method is useful for generating a structured
        cylindrical mesh for VTK (and perhaps other tools).

        Parameters
        ----------
        r : array : The radial values of the grid points.
                    It defaults to linspace(1.0, 2.0, 11).

        theta : array : The angular values of the x axis for the grid
                        points. It defaults to linspace(0,2*pi,11).

        z: array : The values along the z axis of the grid points.
                   It defaults to linspace(0,0,1.0, 11).

        Return
        ------
        points : array
            Nx3 array of points that make up the volume of the annulus.
            They are organized in planes starting with the first value
            of z and with the inside "ring" of the plane as the first
            set of points.  The default point array will be 1331x3.
    """
    # Default values for the annular grid.
    if r is None:
        r = np.linspace(1.0, 2.0, 11)
    if theta is None:
        theta = np.linspace(0, 2 * pi, 11)
    if z is None:
        z = np.linspace(0.0, 1.0, 11)

    # Find the x values and y values for each plane.
    x_plane = (cos(theta) * r[:, None]).ravel()
    y_plane = (sin(theta) * r[:, None]).ravel()

    # Allocate an array for all the points.  We'll have len(x_plane)
    # points on each plane, and we have a plane for each z value, so
    # we need len(x_plane)*len(z) points.
    points = np.empty([len(x_plane) * len(z), 3])

    # Loop through the points for each plane and fill them with the
    # correct x,y,z values.
    start = 0
    for z_plane in z:
        end = start + len(x_plane)
        # slice out a plane of the output points and fill it
        # with the x,y, and z values for this plane.  The x,y
        # values are the same for every plane.  The z value
        # is set to the current z
        plane_points = points[start:end]
        plane_points[:, 0] = x_plane
        plane_points[:, 1] = y_plane
        plane_points[:, 2] = z_plane
        start = end

    return points


# Make the data.
# dims = (51, 25, 25)
# # Note here that the 'x' axis corresponds to 'theta'
# theta = np.linspace(0, 2 * np.pi, dims[0])
# # 'y' corresponds to varying 'r'
# r = np.linspace(1, 10, dims[1])
# z = np.linspace(0, 5, dims[2])
# pts = generate_annulus(r, theta, z)
# Uncomment the following if you want to add some noise to the data.
# pts += np.random.randn(dims[0]*dims[1]*dims[2], 3)*0.04

# s = np.sqrt(pts[:, 0] ** 2 + pts[:, 1] ** 2 + pts[:, 2] ** 2)
# sgrid.point_data.scalars = np.ravel(s.copy())
# sgrid.point_data.scalars.name = "scalars"

# Uncomment the next two lines to save the dataset to a VTK XML file.
# w = tvtk.XMLStructuredGridWriter(input=sgrid, file_name='sgrid.vts')
# w.write()

In [6]:
grid_vtk = grid(ngrid_cart = 30, ngrid_tor = 20, R0 = 2)
sgrid = tvtk.StructuredGrid(dimensions=grid_vtk.tor_x.shape[::-1])
sgrid.points = grid_vtk.torflat

In [ ]:
"""
do the same using unstructured grid just to get 
to know how to work with different grid structures
"""

tet_type = tvtk.Tetra().cell_type # VTK_TETRA == 10
tets = np.arange(len(grid_emc3.cartflat), len(grid_emc3.cartflat)-1)
usgrid_emc3 = tvtk.UnstructuredGrid(points = grid_emc3.cartflat)
usgrid_emc3.set_cells(tet_type, tets)

In [7]:
cell = None
# means not found
cellid = int(4e3)
tol2 = 1e-6
subid = vtk.reference(-1)
pcoord = np.zeros((3))
weights = np.zeros(8)
cid = sgrid.find_cell([2,2,0], cell, cellid, tol2, subid, pcoord, weights)
print(cid, subid, pcoord, weights)

11821 0 [0.87464447 1.         0.78442427] [-1.80013407e-17 -1.25600941e-16  1.88552117e-01  2.70236104e-02
 -6.55022194e-17 -4.57029314e-16  6.86092349e-01  9.83319236e-02]


In [8]:
sg = vtk.vtkStructuredGrid()
# sg.FindCell?
ds = vtk.vtkDataSet.FindCell?

In [9]:
%gui qt

# View the data.
@mayavi2.standalone
def view():
    from mayavi.sources.vtk_data_source import VTKDataSource
    from mayavi.modules.api import Outline, GridPlane

    mayavi.new_scene()
    src = VTKDataSource(data=sgrid)
    mayavi.add_source(src)
    mayavi.add_module(Outline())
    g = GridPlane()
    g.grid_plane.axis = "x"
    mayavi.add_module(g)
    g = GridPlane()
    g.grid_plane.axis = "y"
    mayavi.add_module(g)
    g = GridPlane()
    g.grid_plane.axis = "z"
    mayavi.add_module(g)


if __name__ == "__main__":
    view()

In [14]:
hexahedron_emc3 = sgrid.get_cell(307972)
hexahedron_emc3.points.to_array()

# get the dimension of the cell
hexahedron_emc3.cell_dimension
# get indicies of vertices used in edge id which is the argument
hexahedron_emc3.get_edge_array(4)

hexa_traitnames = hexahedron_emc3.all_trait_names()
type(hexa_traitnames)
type(hexahedron_emc3.points.to_array())

# get the vertexpoints by getting a point object of the hexahedron and casting it to an array
vertex_points = hexahedron_emc3.points.to_array()
vertex_points = np.asarray(vertex_points)
xx, yy, zz = np.meshgrid(vertex_points[:,0], vertex_points[:,1], vertex_points[:,2])
sgobj_hexa = tvtk.StructuredGrid(dimensions = xx.shape[::-1])
sgobj_hexa.points = vertex_points

In [15]:
hexahedron_emc3 = sgrid.get_cell(307972)
points3d = hexahedron_emc3.points.to_array()
points3d.shape
inv = points3d[::-1]
fig, ax = plt.add_subplots(projection='3d')
ax.scatter()

NameError: name 'plt' is not defined

In [ ]:
from mayavi import mlab
# clear the field
mlab.clf()
# mlab.points3d(xx, yy, zz)
mlab.points3d(xx, yy, zz)
# mlab.outline()
mlab.axes()

In [ ]:
# visualize contour of the interpolated values in 3d with colors defined by electron temperature
dimgrid_cart = list(grid_emc3.cart_x.shape)
sgrid_cart_emc3 = tvtk.StructuredGrid(dimensions=dimgrid_cart)
s_Te = pmeshtest.copy()
sgrid_cart_emc3.points = grid_emc3.cartflat
sgrid_cart_emc3.point_data.scalars = s_Te.ravel()
sgrid_cart_emc3.point_data.scalars.name = "Te"

mlab.clf()
#add the dataset to the pipeline
src_cart = mlab.pipeline.add_dataset(sgrid_cart_emc3)
cut_cart = mlab.pipeline.scalar_cut_plane(src_cart)
cont_cart = mlab.pipeline.iso_surface(src_cart)

In [ ]:
"""
A walkthrough on how to plot with the vtk objects
"""

# make the structured grid object from vtk lib
sg_tess = tvtk.StructuredGrid(dimensions=grid_tess.cart_x.shape)
# set the grid point coords using a flattened array of the meshgrid array
sg_tess.points = grid_tess.cartflat
# define a scalar field using the raveled version of the meshgrid array
sg_tess.point_data.scalars = Te_mesh_tessalation.copy().ravel()
# give the scalar parameter a suitable name
sg_tess.point_data.scalars.name = "Te"
# clear the field
mlab.clf()
#add the dataset to the pipeline, the principle is to add several object on top of each other
src_tess = mlab.pipeline.add_dataset(sg_tess)
# make a cut object to cut the 3D volum in slices in a user defined direction
cut_tess = mlab.pipeline.scalar_cut_plane(src_tess)
# make a contour plot of the scalar field
cont_tess = mlab.pipeline.iso_surface(cut_tess)